In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
!pip install feature_engine
from feature_engine.encoding import MeanEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 14.5 MB/s eta 0:00:00


In [ ]:
df_hongos = pd.read_csv("/content/drive/MyDrive/datosTP2/files/MushroomDataset/secondary_data.csv", delimiter=";")

In [ ]:
df_hongos

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61064,p,1.18,s,s,y,f,f,f,f,3.93,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61065,p,1.27,f,s,y,f,f,f,f,3.18,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61066,p,1.27,s,s,y,f,f,f,f,3.86,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,u
61067,p,1.24,f,s,y,f,f,f,f,3.56,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,u


Me quedo con 80% de los datos para train, 10% para validation y 10% para test. 

In [ ]:
y = df_hongos["class"]
X = df_hongos.drop("class", axis = 1)
tamanio_test = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tamanio_test, random_state=9)

In [ ]:
tamanio_validation = 0.1 / (1 - tamanio_test)

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=tamanio_validation, random_state=9)

##**Imputación de nulos**

In [ ]:
df_hongos.nunique()

class                      2
cap-diameter            2571
cap-shape                  7
cap-surface               11
cap-color                 12
does-bruise-or-bleed       2
gill-attachment            7
gill-spacing               3
gill-color                12
stem-height             2226
stem-width              4630
stem-root                  5
stem-surface               8
stem-color                13
veil-type                  1
veil-color                 6
has-ring                   2
ring-type                  8
spore-print-color          7
habitat                    8
season                     4
dtype: int64

Veo cuantos nulos hay por columna

In [ ]:
df_hongos.count()

class                   61069
cap-diameter            61069
cap-shape               61069
cap-surface             46949
cap-color               61069
does-bruise-or-bleed    61069
gill-attachment         51185
gill-spacing            36006
gill-color              61069
stem-height             61069
stem-width              61069
stem-root                9531
stem-surface            22945
stem-color              61069
veil-type                3177
veil-color               7413
has-ring                61069
ring-type               58598
spore-print-color        6354
habitat                 61069
season                  61069
dtype: int64

Según la descripción de los datos, sé que el valor "f" puede usarse tanto para nombrar un dato válido, como para definir un "None". En las columnas donde no sea un dato válido, los voy a reemplazar por "*" que es mi convención para los valores desconocidos.

In [ ]:
columnas_con_variable_f = df_hongos.select_dtypes(include=["object"]).columns.map(lambda x: x if df_hongos[x].str.contains("f").any() else None).dropna() 
columnas_con_variable_f

Index(['cap-shape', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing',
       'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'has-ring',
       'ring-type'],
      dtype='object')

###**Nulos en columnas de gill-":**

In [ ]:
columnas_gill =  ["gill-attachment", "gill-spacing", "gill-color"]

In [ ]:
df_hongos.loc[ : , columnas_gill ]   

,gill-attachment,gill-spacing,gill-color
0,e,NaN,w
1,e,NaN,w
2,e,NaN,w
3,e,NaN,w
4,e,NaN,w
...,...,...,...
61064,f,f,f
61065,f,f,f
61066,f,f,f
61067,f,f,f


In [ ]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-attachment"] == "f"] 

,gill-attachment,gill-spacing,gill-color
54362,f,f,f
54363,f,f,f
54364,f,f,f
54365,f,f,f
54366,f,f,f
...,...,...,...
61064,f,f,f
61065,f,f,f
61066,f,f,f
61067,f,f,f


In [ ]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-attachment"] == "f"].nunique() 

gill-attachment    1
gill-spacing       1
gill-color         1
dtype: int64

In [ ]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-spacing"] == "f"].nunique() 

gill-attachment    1
gill-spacing       1
gill-color         1
dtype: int64

In [ ]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-color"] == "f"].nunique() 

gill-attachment    1
gill-spacing       1
gill-color         1
dtype: int64

En lo anterior puedo ver que las columnas con gill son siempre "f" todas a la vez. Supongo así que el gill no es desconocido, sino que no lo tiene el hongo. \\
Entonces dejo esa variable "f" tal como está en esas columnas porque me da información.

###**Nulos en columnas de stem- :**

In [ ]:
df_hongos["stem-height"].sort_values(ascending=True)

57818     0.00
56566     0.00
56565     0.00
56564     0.00
56563     0.00
         ...  
2881     32.85
2897     33.03
2937     33.25
3143     33.72
2870     33.92
Name: stem-height, Length: 61069, dtype: float64

In [ ]:
df_hongos["stem-width"].sort_values(ascending=True)

57948      0.00
57845      0.00
57844      0.00
57843      0.00
57842      0.00
          ...  
48501     95.68
48447     98.00
48366    101.69
48682    102.48
48597    103.91
Name: stem-width, Length: 61069, dtype: float64

In [ ]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ df_hongos["stem-height"] == 0.00] 

,stem-root,stem-surface,stem-color
56480,f,f,f
56481,f,f,f
56482,f,f,f
56483,f,f,f
56484,f,f,f
...,...,...,...
58240,f,f,f
58241,f,f,f
58242,f,f,f
58243,f,f,f


In [ ]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ df_hongos["stem-width"] == 0.00] 

,stem-root,stem-surface,stem-color
56480,f,f,f
56481,f,f,f
56482,f,f,f
56483,f,f,f
56484,f,f,f
...,...,...,...
58240,f,f,f
58241,f,f,f
58242,f,f,f
58243,f,f,f


In [ ]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ (df_hongos["stem-height"] == 0.00) & (df_hongos["stem-width"] == 0.00)]

,stem-root,stem-surface,stem-color
56480,f,f,f
56481,f,f,f
56482,f,f,f
56483,f,f,f
56484,f,f,f
...,...,...,...
58240,f,f,f
58241,f,f,f
58242,f,f,f
58243,f,f,f


In [ ]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ (df_hongos["stem-height"] == 0.00) & (df_hongos["stem-width"] == 0.00)].nunique()

stem-root       1
stem-surface    1
stem-color      1
dtype: int64

In [ ]:
df_hongos.loc[(df_hongos["stem-height"] == 0.00) & (df_hongos["stem-width"] == 0.00)].head(3)

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
56480,p,2.48,o,t,n,f,NaN,c,w,0.0,...,f,f,f,NaN,NaN,f,f,n,d,u
56481,p,4.29,o,t,w,f,NaN,c,w,0.0,...,f,f,f,NaN,NaN,f,f,n,d,u
56482,p,4.29,o,t,n,f,NaN,c,w,0.0,...,f,f,f,NaN,NaN,f,f,n,d,u


Veo que cuando no tienen valor los tallos de los hongos, el valor es siempre "f", que lo supongo desconocido. \\
En el paper decía que secondary_data solo tenía hongos con tallo, por lo que voy a remplazar los valores de tallos inválidos por la media de la columna en todo el dataset, sabiendo que solo valen 0.00 los valores de altura y ancho del tallo para ambos a la vez.

In [ ]:
promedio_alto_tallo_dataset = df_hongos["stem-height"].mean()
promedio_ancho_tallo_dataset = df_hongos["stem-width"].mean()


X_train["stem-height"] = X_train["stem-height"].replace(0.00, promedio_alto_tallo_dataset)
X_validation["stem-height"] = X_validation["stem-height"].replace(0.00, promedio_alto_tallo_dataset)
X_test["stem-height"] = X_test["stem-height"].replace(0.00, promedio_alto_tallo_dataset)

X_train["stem-width"] = X_train["stem-width"].replace(0.00, promedio_ancho_tallo_dataset)
X_validation["stem-width"] = X_validation["stem-width"].replace(0.00, promedio_ancho_tallo_dataset)
X_test["stem-width"] = X_test["stem-width"].replace(0.00, promedio_ancho_tallo_dataset)

###**Nulos en columna has-ring y ring-type:**

In [ ]:
df_hongos["has-ring"].unique()

array(['t', 'f'], dtype=object)

In [ ]:
df_hongos["ring-type"].loc[ df_hongos["has-ring"] == "f" ].unique()

array(['f'], dtype=object)

In [ ]:
df_hongos["ring-type"].loc[ df_hongos["has-ring"] == "t" ].unique()

array(['g', 'p', 'e', 'l', 'm', nan, 'r', 'f', 'z'], dtype=object)

In [ ]:
df_hongos["ring-type"].loc[ (df_hongos["has-ring"] == "t") & (df_hongos["ring-type"] == "f") ]

30005    f
30006    f
30007    f
30008    f
30009    f
        ..
59652    f
59653    f
59654    f
59655    f
59656    f
Name: ring-type, Length: 2471, dtype: object

Veo en lo anterior que el "f" de ring-type y has-ring lo puedo suponer como desconocido. Esto porque veo que no significa que no tenga anillo por ejemplo.

###**Nulos en columna veil-type:**

In [ ]:
df_hongos["veil-type"].unique()

array(['u', nan], dtype=object)

Supongo que los NaN de "veil-type" son el otro valor posible ("p").

In [ ]:
X_train["veil-type"] = X_train["veil-type"].fillna("p")
X_validation["veil-type"] = X_validation["veil-type"].fillna("p")
X_test["veil-type"] = X_test["veil-type"].fillna("p")

###**Asignación de constante a valores nan y "f" desconocidos:**

Remplazo todos los NaN restantes por *. Esto porque en el texto y en los gráficos no encontré nada para suponer otra cosa. \\
Uso * porque supongo que no lo van a pasar nunca como dato de una columna.



In [ ]:
X_train = X_train.fillna("*")
X_validation = X_validation.fillna("*")
X_test = X_test.fillna("*")

También reemplazo todos los "f" que supongo desconocidos por el mismo "*" para tener una convención más uniforme para los desconocidos o nulos. \\
No uso "f" porque otras columnas lo usan con un significado real.

In [ ]:
columnas_con_f_desconocidos = ["stem-root", "stem-surface", "stem-color", "has-ring", "ring-type"]


X_train[columnas_con_f_desconocidos] = X_train[columnas_con_f_desconocidos].replace("f", "*")
X_validation[columnas_con_f_desconocidos] = X_validation[columnas_con_f_desconocidos].replace("f", "*")
X_test[columnas_con_f_desconocidos] = X_test[columnas_con_f_desconocidos].replace("f", "*")

##**Encoding:**

Me quedo para MeanEncoding las columnas con más de 7 variables categóricas diferentes, para ahorrarme que se encodeen en muchas columnas con OHE.

In [ ]:
cols_categoricas = df_hongos.select_dtypes(include=["object"]).columns.drop("class")
cols_para_mean_encoding = df_hongos[cols_categoricas].columns.map(lambda x: x if df_hongos[x].nunique() > 7 else None).dropna()
columnas_categoricas_para_ohe = cols_categoricas.drop(cols_para_mean_encoding)

In [ ]:
one_hot_encoder = OneHotEncoder(sparse_output=False, drop="if_binary", dtype=int)

X_train_encoded = one_hot_encoder.fit_transform( X_train[columnas_categoricas_para_ohe] )
X_validation_encoded = one_hot_encoder.transform( X_validation[columnas_categoricas_para_ohe] ) 
X_test_encoded = one_hot_encoder.transform( X_test[columnas_categoricas_para_ohe] ) 

In [ ]:
nombres_features = one_hot_encoder.get_feature_names_out(columnas_categoricas_para_ohe)

columnas_numericas = X_train[["cap-diameter", "stem-height", "stem-width"]]
X_train_encoded = pd.DataFrame(X_train_encoded, index=X_train.index, columns=nombres_features)
X_train_encoded = pd.concat([X_train_encoded, columnas_numericas], axis=1)

columnas_numericas = X_validation[["cap-diameter", "stem-height", "stem-width"]]
X_validation_encoded = pd.DataFrame(X_validation_encoded, index=X_validation.index, columns=nombres_features)
X_validation_encoded = pd.concat([X_validation_encoded, columnas_numericas], axis=1)

columnas_numericas = X_test[["cap-diameter", "stem-height", "stem-width"]]
X_test_encoded = pd.DataFrame(X_test_encoded, index=X_test.index, columns=nombres_features)
X_test_encoded = pd.concat([X_test_encoded, columnas_numericas], axis=1)

In [ ]:
y_train_encoded = y_train.map(lambda x: 0 if x == 'e' else 1)
y_validation_encoded = y_validation.map(lambda x: 0 if x == 'e' else 1)
y_test_encoded = y_test.map(lambda x: 0 if x == 'e' else 1)

In [ ]:
mean_encoder = MeanEncoder()

X_train_mean_enc = mean_encoder.fit_transform( X_train[cols_para_mean_encoding], y_train_encoded )
X_validation_mean_enc = mean_encoder.transform( X_validation[cols_para_mean_encoding] )
X_test_mean_enc = mean_encoder.transform( X_test[cols_para_mean_encoding] )

In [ ]:
X_train_encoded = pd.concat([X_train_encoded, X_train_mean_enc], axis=1)
X_validation_encoded = pd.concat([X_validation_encoded, X_validation_mean_enc], axis=1)
X_test_encoded = pd.concat([X_test_encoded, X_test_mean_enc], axis=1)

In [ ]:
X_train_encoded.head(3)

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_o,cap-shape_p,cap-shape_s,cap-shape_x,does-bruise-or-bleed_t,gill-attachment_*,gill-attachment_a,...,cap-diameter,stem-height,stem-width,cap-surface,cap-color,gill-color,stem-surface,stem-color,ring-type,habitat
57811,0,0,0,1,0,0,0,0,0,0,...,4.76,6.581538,12.14941,0.426332,0.477941,0.602261,0.521569,1.000000,0.549975,0.544559
47496,1,0,0,0,0,0,0,0,0,1,...,3.60,8.490000,2.55000,0.527300,0.598258,0.612028,0.521569,0.597432,0.549975,0.684417
58325,0,0,0,1,0,0,0,0,0,0,...,53.18,6.360000,32.86000,0.472211,0.581613,0.591117,0.671962,0.774290,0.549975,0.544559


Veo que hay 56 features.

##**Modelo:**

In [ ]:
xgb_classifier = xgb.XGBClassifier(seed=9)

In [ ]:
xgb_classifier.fit(X_train_encoded, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
from sklearn.metrics import roc_auc_score

Veo que no fue necesario buscar hiperparámetros para llegar al auc-roc de 1.0.

In [ ]:
roc_auc_score(y_validation_encoded, xgb_classifier.predict_proba(X_validation_encoded)[:, 1])

1.0

**¿Cuál es el score en test? (guardar el csv con predicciones para entregarlo después)**

Finalmente, habiendo llegado al modelo final, mido para test:

In [ ]:
roc_auc_score(y_test_encoded, xgb_classifier.predict_proba(X_test_encoded)[:, 1])

1.0

In [ ]:
predicciones = xgb_classifier.predict(X_test_encoded)
predicciones = pd.DataFrame(predicciones, index=X_test_encoded.index, columns=["e"])
path = '/content/drive/MyDrive/datosTP2/files/MushroomDataset/predicciones_xgb.csv'
predicciones.to_csv(path)

https://drive.google.com/file/d/1q4gNtdlslYvpi60N3pMhF8IgUNNunAav/view?usp=sharing

Ya se mostró como buscar hiperparámetros en el baseline.

-----------------------

**¿Por qué cree que logro/no logro el mismo valor de AUC que con Random Forest?** \\
Logré el mismo valor que con Random Forest creo que porque son ambos modelos basados en árboles de decisión y este problema de clasificación se amoldaba fácilmente. \\
Sino como es un resultado muy bueno mean encoding puede haber leakeado data, pero como RF también daba igual no creo.


-----------------------

Si tuviese un score menor y quisiera buscar podría hacer algo como lo siguiente, buscando para los hiperparámetros más importantes:



In [ ]:
params = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [5, 7, 10],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.3, 0.4],
    'n_estimators': [50, 100, 200],
    'objective': ['binary:logistic'],
    'gamma': [0.1, 0.3], 
    'alpha': [0.4, 0.7],
    'lambda': [0.1, 1]
  }

In [ ]:
randomized_search = RandomizedSearchCV(xgb_classifier, param_distributions=params, n_iter=5, random_state=9, scoring='roc_auc')

In [ ]:
randomized_search.fit(X_train_encoded, y_train_encoded)

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None...
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=5,
                   param_distributions={'alpha': [0.4, 0.7],
                                        'colsample_bytree': [0.3, 0.4],
                                        'gamma': [0.1, 0.3], 'lambda': [0.1, 1],
                                        'learning_rate': [0.1, 0.2],
                                        'max_depth': [5, 7, 10],
                                        'n_estimators': [50, 100, 200],
                                        'objective': ['binary:logistic'],
                                        'subsample': [0.7, 0.8]},
                   random_state=9, scoring='roc_auc')

In [ ]:
randomized_search.best_params_

{'subsample': 0.8,
 'objective': 'binary:logistic',
 'n_estimators': 100,
 'max_depth': 10,
 'learning_rate': 0.2,
 'lambda': 1,
 'gamma': 0.1,
 'colsample_bytree': 0.4,
 'alpha': 0.7}

In [ ]:
xgb_model = randomized_search.best_estimator_

Entreno con el modelo con los mejores hiperparámetros encontrados.

In [ ]:
xgb_model.fit(X_train_encoded, y_train_encoded)

XGBClassifier(alpha=0.7, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=1, learning_rate=0.2,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
roc_auc_score(y_validation_encoded, xgb_model.predict_proba(X_validation_encoded)[:, 1])